In [7]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import requests
from tqdm import tqdm_notebook as tqdm
import time
import random

from bs4 import BeautifulSoup
from copy import deepcopy

In [8]:
# boot up scraper for chrome
driver = webdriver.Chrome()

In [314]:
# go to ameriprise website
driver.get('https://www.ameripriseadvisors.com/')

# Isolate each box/container

In [317]:
# use beautiful soup to load website information
soup = BeautifulSoup(driver.page_source)

In [318]:
# match number of ameriprise advisors
len(soup.find_all('div',{'class':'right-pane'}))

254

In [319]:
# save each container to a list
advisor_list = soup.find_all('div',{'class':'right-pane'})

## Get Name

In [35]:
# code to get name
advisor_list[0].find('h3').text

'Vasso & Associates'

## Get Titles

In [73]:
# find advisor title: some are more important that others
advisor_list[1].find('p', {'class':'title'}).text

'Private Wealth Advisor\n • Vasso & Associates\n'

In [315]:
# # iterate through advisor list to check for mistakes
# # spits out '\n'
# for i in advisor_list:
#     print(i.find('h3').text)
#     print(i.find('p', {'class':'title'}).text)
#     print('\n')

### Team affiliation is after * charachter

In [75]:
# find team that advisor is on
advisor_list[1].find('p', {'class':'title'}).text

'Private Wealth Advisor\n • Vasso & Associates\n'

## Phone

In [76]:
# find phone number
advisor_list[1].find('a', {'class':'phone-link'}).text

'972.692.5069'

## Email

In [92]:
# find email address
advisor_list[1].find('p', {'class':'email'}).find('a', href=True)['href']

'mailto:neal.d.vasso@ampf.com'

In [95]:
# # test 
# for i in advisor_list:
#     print(i.find('p', {'class':'email'}).find('a', href=True)['href'])

## Address

In [101]:
# find address
advisor_list[1].find('a', {'class':'address'}).text

'\n5000 Quorum Dr Ste 375\n\nDallas, TX 75254-7091\n'

In [105]:
# # test 
# for i in advisor_list:
#     print(i.find('a', {'class':'address'}).text)
#     print('\n')

## Link to website

In [107]:
# find link to website
advisor_list[1].find('a', href=True)['href']


'/neal.d.vasso/?awsrc='

In [109]:
# # test 
# for i in advisor_list:
#     print(i.find('a', href=True)['href'])
#     print('\n')

# sample pipeline

In [361]:
# # create a list of advisors
# list_of_advisor_dicts = []

In [362]:
# save page information
soup = BeautifulSoup(driver.page_source)

In [363]:
# create list of advisors from page
advisor_list = soup.find_all('div',{'class':'right-pane'})

In [364]:
# match number of advisors to result number
len(advisor_list)

30

In [365]:
# save each part of what we found into the dictionaries
for i in advisor_list:
    advisor = {}
    try:
        advisor['name'] = i.find('h3').text
    except:
        advisor['name'] = None
    try:
        advisor['title'] = i.find('p', {'class':'title'}).text
    except:
        advisor['title'] = None
    
    try:
        advisor['phone'] = i.find('a', {'class':'phone-link'}).text
    except: 
        advisor['phone'] = None
    
    try:
        advisor['email'] = i.find('p', {'class':'email'}).find('a', href=True)['href']
    except:
        advisor['email'] = None
    
    try:
        advisor['address'] = i.find('a', {'class':'address'}).text
    except:
        advisor['address'] = None
    
    try:
        advisor['link_to_website'] = i.find('a', href=True)['href']
    except:
        advisor['link_to_website'] = None
    
    list_of_advisor_dicts.append(advisor)

In [366]:
# how many advisors are on the list now
len(list_of_advisor_dicts)

1241

In [367]:
# save into dataframe
df = pd.DataFrame(list_of_advisor_dicts)

## Save raw into csv

In [368]:
#save original to csv
raw_df = deepcopy(df)

In [369]:
raw_df.shape

(1241, 6)

In [370]:
raw_df.to_csv('ameriprise_morski_raw.csv')

## Drop duplicates

In [371]:
df.head()

,name,title,phone,email,address,link_to_website
0,Fields Wealth Management,\n,817.591.8000,None,"\n1401 Precinct Line Rd\n\nHurst, TX 76053-3827\n",/team/fields-wealth-management/?awsrc=
1,Charla K McIntyre Fields,Private Wealth Advisor\n • Fields Wealth Manag...,817.591.8002,mailto:charla.k.mcintyre-fields@ampf.com,"\n1401 Precinct Line Rd\n\nHurst, TX 76053-3827\n",/charla.k.mcintyre-fields/?awsrc=
2,Vasso & Associates,\n,972.692.5069,mailto:neal.d.vasso@ampf.com,"\n5000 Quorum Dr Ste 375\n\nDallas, TX 75254-7...",/team/vasso-associates/?awsrc=
3,Neal D Vasso,Private Wealth Advisor\n • Vasso & Associates\n,972.692.5069,mailto:neal.d.vasso@ampf.com,"\n5000 Quorum Dr Ste 375\n\nDallas, TX 75254-7...",/neal.d.vasso/?awsrc=
4,Doyle & Associates,\n,214.389.8500,mailto:terry.d.doyle@ampf.com,"\n2435 N Central Expy\nSte 915\nRichardson, TX...",/team/doyle-associates/?awsrc=


In [373]:
#drop duplicate values if they have the same name and email
df = df.sort_values('name').drop_duplicates(subset=['name','email'], keep = 'last')

## Fix Columns

In [374]:
# get title from title column
df['correct_title'] = df['title'].str.split('\n', expand = True)[0]

In [375]:
# get team from title column
df['team'] = df['title'].str.split('\n', expand = True)[1]

In [376]:
# get rid of phrase 'mail to:' in email column
df['email'] = df['email'].str[7:]

In [377]:
# get city, state, and zip from address column
df['city, state, zip'] = df['address'].str.split('\n', expand =True)[3]

In [378]:
# get the primary address
df['address1'] = df['address'].str.split('\n', expand =True)[1]

In [379]:
# get suite number from address
df['address2'] = df['address'].str.split('\n', expand =True)[2]

In [380]:
# get rid of phrase from the link column
df['link_to_website'] = df['link_to_website'].str[:-8]

In [381]:
# create a column with the proper address
df['link'] = 'https://www.ameripriseadvisors.com/'

In [382]:
# concatenate link and link_to_website to have the right web address
df['link'] = df['link']+df['link_to_website'].astype(str)

In [383]:
# isolate city from city,state, zip column
df['city'] = df['city, state, zip'].str.split(', ', expand=True)[0]

In [384]:
# get zip code from city, state, and zip column
df['zip'] = df['city, state, zip'].str.split(', ', expand=True)[1]

In [385]:
# strip the first 3 characters from zipcode
df['zip']= df['zip'].str[3:]

In [386]:
# strip the first 3 charachters from team 
df['team'] = df['team'].str[3:]

In [387]:
# create the right order for dataframe
new_cols = ['name', 'team', 'phone', 'email', 'address1', 'address2', 'city', 'zip', 'link', 'correct_title']

In [388]:
# save into new dataframe
df = df[new_cols]

In [389]:
# sort df by city, then, address, then team
df = df.sort_values(by=['city', 'address1', 'team'])

In [390]:
# reset index
df.reset_index(inplace=True)

In [391]:
df.head()

,index,name,team,phone,email,address1,address2,city,zip,link,correct_title
0,1021,Watters Creek Wealth Management,,972.954.1595,clint.w.wammack@ampf.com,700 Central Expy S,Ste 380,Allen,75013-8110,https://www.ameripriseadvisors.com//team/watte...,
1,413,Clint Wammack,Watters Creek Wealth Management,972.954.1595,clint.w.wammack@ampf.com,700 Central Expy S,Ste 380,Allen,75013-8110,https://www.ameripriseadvisors.com//clint.w.wa...,Financial Advisor
2,700,Jenny Funderburk,Watters Creek Wealth Management,972.954.1595,Jenny.A.Funderburk@ampf.com,700 Central Expy S,Ste 380,Allen,75013-8110,https://www.ameripriseadvisors.com//jenny.a.fu...,Financial Advisor
3,336,Zachary Swain,,817.427.5300,zachary.g.swain@ampf.com,2000 E Lamar Blvd,Ste 600,Arlington,76006-7340,https://www.ameripriseadvisors.com//zachary.g....,Private Wealth Advisor
4,573,Tre Carruthers,Carruthers & Associates,682.323.8429,tre.l.carruthers@ampf.com,2221 E Lamar Blvd,Ste 220,Arlington,76006-7418,https://www.ameripriseadvisors.com//tre.l.carr...,Financial Advisor


In [392]:
# drop auto created columns from reset_index
df.drop(columns = ['index'], inplace=True)

In [393]:
# save
df.to_csv('morski_ameriprise_fixed2.csv')